In [19]:
import pyautogui as pg
import time

In [21]:
NEXT_PAGE = (540, 608)
EXPORT_BUTTON = (696, 568)
EXCEL = (650, 789)
RECORD_CONTENT_BUTTON = (632, 891)
FULL_EXPORT = (388, 945)
SEQ = [EXPORT_BUTTON, EXCEL, RECORD_CONTENT_BUTTON, FULL_EXPORT, FULL_EXPORT, NEXT_PAGE]

In [ ]:
def download():
    while True:
        for i in SEQ:
            pg.click(*i)
            time.sleep(2)
#download()